<a href="https://colab.research.google.com/github/Fjallripa/TinyStories/blob/main/tiny_stories_test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Use the Colab UI instead. Select the folder button on the left and then the drive button.
# This way you will not be asked every execution for permissions
from google.colab import drive
drive.mount('/content/drive')

!pip install -q datasets

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00


In [2]:
from transformers import GPTNeoConfig, GPTNeoForCausalLM, get_scheduler, \
    DataCollatorForSeq2Seq, AutoTokenizer, DataCollatorForLanguageModeling, \
    AutoModelForCausalLM, AutoConfig
from datasets import load_dataset, DatasetDict, Dataset, concatenate_datasets, \
    Sequence, Value
import torch
from torch.utils.data import DataLoader
from datetime import datetime
from copy import deepcopy
from tqdm.auto import tqdm
import numpy as np
import os
import matplotlib.pyplot as plt
import re
import matplotlib.pyplot as plt

# device
device = "cuda"  if torch.cuda.is_available() else  "cpu"
'''
    if torch.cuda.is_available():
        device = torch.device("cuda")  # GPU
    else:
        device = torch.device("cpu")  # CPU
'''
print("Selected device:", device)

# language
lang = "en" # Can be "en" or "de"
print(f"Selected language: {lang}")


Selected device: cpu
Selected language: en


In [3]:
tokenizer_en = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer = AutoTokenizer.from_pretrained("dbmdz/german-gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/865 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.43M [00:00<?, ?B/s]

In [8]:
print((len(tokenizer) - len(tokenizer_en))/len(tokenizer_en))
print(len(tokenizer))
print(len(tokenizer_en))

0.00017907953120958273
50266
50257


In [24]:
r = range(257, 270)
print(" & ".join([str(i) for i in r]))
print(" & ".join([f"\"{tokenizer_en.decode([i])}\"" for i in r]))

257 & 258 & 259 & 260 & 261 & 262 & 263 & 264 & 265 & 266 & 267 & 268 & 269
" a" & "he" & "in" & "re" & "on" & " the" & "er" & " s" & "at" & " w" & " o" & "en" & " c"


In [3]:
model = GPTNeoForCausalLM.from_pretrained(f"/content/drive/My Drive/genAI project/model-33m-de-2024-03-23T01:23:36.822703")
#input_ids = tokenizer.encode(f"Once upon a time Tom ate apples and", return_tensors="pt")
input_ids = tokenizer.encode(f"Eines Tages traf Lilly ein Auto und", return_tensors="pt")
output = model.generate(input_ids, max_length = 1024, num_beams=5, temperature=3, repetition_penalty=3.0, eos_token_id = 50256)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

NameError: name 'GPTNeoForCausalLM' is not defined

In [10]:

print(tokenizer.decode(output[0], skip_special_tokens=False))

Eines Tages traf Lilly ein Auto und fragte: „Wo gehst du hin?" Das Auto antwortete: „Ich gehe in den Park, um mit meinen Freunden zu spielen. Willst du mit mir kommen?" Lilly war aufgeregt und sagte: „Ja! Lass uns gehen!" Also fuhren sie zusammen zum Park. Als sie dort ankamen, sahen sie eine große Rutsche. Lilly stieg die Leiter hinauf und setzte sich auf die Rutsche. Sie lachte und sagte: „Das ist so viel Spaß!" Nachdem sie eine Weile gespielt hatten, gingen sie zur Rutsche. Lilly kletterte die Leiter hinauf und als sie oben ankam, schrie sie vor Freude: „Das war so viel Spaß!"
Es war einmal ein kleines Mädchen namens Lily. Sie liebte es, draußen im Garten mit ihren Spielzeugen zu spielen. Eines Tages bat Lilys Mutter sie, ihr Spielzeug aufzuräumen. Lily wollte nicht aufräumen, aber sie wusste, dass es wichtig war, ihrer Mutter zu helfen. Später an diesem Tag kam Lilys Vater von der Arbeit nach Hause. Er sah das Chaos und sagte: "Lily, kannst du mir bitte helfen, dein Spielzeug aufzu

In [ ]:
preprocessed_datasets_path = "/content/drive/My Drive/genAI project/tiny-stories-de-preprocessed"
ds = DatasetDict.load_from_disk(preprocessed_datasets_path)
print(ds["train"][0]["input_ids"])
print(ds["train"][0]["labels"])

In [2]:

print(ds["train"][9]["input_ids"])

NameError: name 'ds' is not defined

In [3]:
losses1 = torch.load(f"/content/drive/My Drive/genAI project/losses_tensor-1m-en-2024-03-22T09:06:10.954130.pt")
losses2 = torch.load(f"/content/drive/My Drive/genAI project/losses_tensor-1m-en-2024-03-21T12:49:59.608517.pt")

NameError: name 'torch' is not defined

In [4]:
losses = list(losses2)
losses.extend(list(losses1))
print(len(losses))

NameError: name 'losses2' is not defined

In [5]:
plt.plot(losses)

NameError: name 'plt' is not defined